# Masterlist overview

This notebook summarises some important properties of the data presented here. This includes

- The area covered by each field
- The number of total sources in each field
- The number of bands which we have data for in each field
- The size of the masterlist in each field

In [1]:
from astropy.table import Table, Column
from astropy import units as u
import numpy as np

from pymoc import MOC

import os
#import humanfriendly

In [2]:
dr1 = [
# Field                suffix
['AKARI-NEP',          '20171202' ],
['AKARI-SEP',          '20180122' ],
['Bootes',             '20180111' ],
['CDFS-SWIRE',         '20180129' ],
['COSMOS',             '20180112' ],
['EGS',                '20171202' ],
['ELAIS-N1',           '20171016' ],
['ELAIS-N2',           '20171207' ],
['ELAIS-S1',           '20180122' ],
['GAMA-09',            '20171206' ],
['GAMA-12',            '20180129' ],
['GAMA-15',            '20180129' ],
['HDF-N',              '20171212' ],
['Herschel-Stripe-82', '20180113' ],
['Lockman-SWIRE',      '20171201' ],
['NGP',                '20171207' ],
['SA13',               '20171123' ],
['SGP',                '20180124' ],
['SPIRE-NEP',          '20171122' ],
['SSDF',               '20180122' ],
['xFLS',               '20171201' ],
['XMM-13hr',           '20171123' ],
['XMM-LSS',            '20180130' ]]

In [3]:
masterlist_summary = Table()
masterlist_summary.add_column(Column(data = np.array(dr1)[:,0], name = 'field'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'suffix'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'objects'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), np.nan, dtype=float), name = 'area'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), np.nan, dtype=float), name = 'file_size'))

In [4]:
help_moc = MOC(filename='../dmu2/help_coverage_MOC.fits')
print('Total HELP area: {} sqare degrees'.format(round(help_moc.area_sq_deg)))

Total HELP area: 1270 sqare degrees


In [5]:
for field in dr1:
    number_objects = len(Table.read('dmu1_ml_{}/data/master_list_cross_ident_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])))
    masterlist_summary['objects'][masterlist_summary['field'] == field[0]] = number_objects
    
    
    moc = MOC(filename='../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field[0]))
    area = round(moc.area_sq_deg)
    masterlist_summary['area'][masterlist_summary['field'] == field[0]] = area
    
    masterlist_summary['suffix'][masterlist_summary['field'] == field[0]] = field[1]
    
    size = os.stat('dmu1_ml_{}/data/master_catalogue_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])).st_size
    #Can't install packages on Vulcain
    #size_readable = humanfriendly.format_size(size)
    masterlist_summary['file_size'][masterlist_summary['field'] == field[0]] = size
    
    #TODO:
    num_surveys = 0.
    
    num_bands = 0.

In [6]:
masterlist_summary.show_in_notebook()

idx,Field,#Objects,#Area (sq. degrees),#File size
0,AKARI-NEP,285786,9.0,241231680.0
1,AKARI-SEP,844172,9.0,524255040.0
2,Bootes,3388482,11.0,4537221120.0
3,CDFS-SWIRE,2171051,13.0,5731646400.0
4,COSMOS,2779194,5.0,4630199040.0
5,EGS,1412613,4.0,3890416320.0
6,ELAIS-N1,4026292,14.0,5242279680.0
7,ELAIS-N2,1783240,9.0,1941984000.0
8,ELAIS-S1,1655564,9.0,1908901440.0
9,GAMA-09,12937982,62.0,19652852160.0


In [2]:
print('Total number of objects in HELP: {}'.format(np.sum(masterlist_summary['Objects'])))

Total number of objects in HELP: 170619898


In [7]:
masterlist_summary.write('dr1_overview.fits', overwrite=True)